In [1]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import Aer, BasicAer, transpile
from math import pi, sqrt

In [2]:
def inv_qft_gate(nbdigits):
    qr = QuantumRegister(nbdigits, 'qr')
    circuit = QuantumCircuit(qr)
    for i in range(nbdigits // 2):
        circuit.swap(i, nbdigits-i-1)
    for i in range(nbdigits):
        circuit.h(qr[i])
        for j in range(i+1, nbdigits):
            rot_phase = - 2*pi/2**(j-i+1)
            circuit.cp(rot_phase, j, i)
    return circuit.to_gate()

In [3]:
nbdigits = 5

In [4]:
initial_state = np.zeros(2**(nbdigits+1), dtype=np.complex_)
# initial_state[0] = 1. / sqrt(2)
initial_state[2**nbdigits] = 1.

In [8]:
from qiskit.circuit.library import PhaseGate

theta = 0.5+0.125
gate = PhaseGate(2*pi*theta)

In [9]:
circuit = QuantumCircuit(nbdigits+1, nbdigits)
controlled_gate = gate.control(1)
circuit.initialize(initial_state)
circuit.h(range(nbdigits))
for i in range(nbdigits):
    for _ in range(2**(i)):
        circuit.append(controlled_gate, [i, nbdigits])
        # circuit.cp(2*pi*theta, i, nbdigits)
circuit.append(inv_qft_gate(nbdigits), range(nbdigits))
circuit.measure(range(nbdigits), range(nbdigits))

In [10]:
backend = Aer.get_backend('aer_simulator')
tqc = transpile(circuit, backend)
job = backend.run(tqc, shots=1000)
result = job.result()
counts = result.get_counts()
print(counts)

{'10100': 1000}
